## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [45]:
import pandas as pd

Прочитаем файл */datasets/data.csv* и сохраним его в переменной *data*.

In [46]:
data = pd.read_csv('/datasets/data.csv')
data.head(10) # Получение первых 10 строк таблицы.

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Общая информация о данных таблицы *data*.

In [47]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Каждая строка таблицы содержит информацию о клиенте банка. Различное значение в столбцах говорит о том, что в данных есть отсутствующие значения. Столбец *education* содержит значения с разным регистром. В столбце *days_employed* невооруженным глазом видны артефакты. Для проверки гипотиз особенно ценны столбцы *children*, *family_status*, *total_income*, *purpose* и *debt*.

### Шаг 2. Предобработка данных

### Обработка пропусков

Проверяем данные на наличие пропусков.

In [48]:
data.isnull().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Столбец *days_employed* состоит либо из отрицательных, либо из черезмерно больших значений. 
Ввиду того, что в данном проекте значения из этого столбца не представляют интереса, 
оставим их без изменения, убрав пропуски.
Избавимся от пропусков в данных столбцах и заменем вещественный тип данных на целочисленный.

In [49]:
data['days_employed'] = data['days_employed'].fillna('0')


Пропуски в столбце *total_income* заменим медианой ежемесячного дохода.

In [50]:
total_median = data['total_income'].median()
data['total_income'] = data['total_income'].fillna(total_median)

Убедимся, что избавились от пропусков.

In [51]:
data.isnull().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Заменяем вещественный тип данных на целочисленный.

### Вывод

На данном этапе были обнаружены пропуски в двух столцах. так как стобец о ежемесячном доходе представляет непосредственную важность для исследовани мы не удалили, а заполнили его пропущенные значения.
Заметим, что количество пропусков в столбце days_employed было равно количеству пропусков в столбце total_income. Эти пропуски неслучайные. Можно предположить, что часть клиентов банка не работет, работает неофициально.

### Замена типа данных

Количество трудовых дней, как и ежемесячный доход не являются целыми числами, исправим это.

In [52]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

С помощью метода **astype()** данные столбцов *days_employed* и *total_income* привели к виду *float64*. Типы данных остальных столбцов являются верными.

### Обработка дубликатов

In [53]:
# Столбцы, в которых возможен разный регистр приведем к нижнему.
data['education'] = data['education'].str.lower() 
data['family_status'] = data['family_status'].str.lower()
data['income_type'] = data['income_type'].str.lower()
data['purpose'] = data['purpose'].str.lower()

# Узнаем количество дубликатов всей таблицы.
data.duplicated().sum()

71

In [54]:
data.drop_duplicates().reset_index(drop=True)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,-4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21450,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21451,1,-2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21452,3,-3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


### Вывод

Сперва с помощью **str_lower()** привели к нижнему регистру все значения в столбце *education* и подсчитали количество дубликатов в таблице. Так как у нас нет ID заемщика, мы не можем быть полностью уверены, что дубликаты - это одни и те же люди, но так как вероятность совпадения значения в каждом поле очень мала, то мы можем полагать, что это дубликаты, и благодаря **drop_duplicates()** можем избавиться от них.

### Лемматизация

In [55]:
# Лемматизировать будем столбец с целями кредита и создадим для него новый столбец.
from pymystem3 import Mystem
m = Mystem()

def lemma(row):
    return m.lemmatize(row)

data['lemma_purpose'] = data['purpose'].apply(lemma)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


### Вывод

Написали функцию, которая для каждой строки возвращает лемматизированный вариант и применили такую функцию через метод **apply**, получив тем самым столбце с лемматизацией для целей кредита. После лемматизации можно выделить определенный ТОП запросов на получение кредита.

### Категоризация данных

In [56]:
total = data['debt'].sum() # Общее количес
total

1741

In [57]:
data['children'].value_counts() # Уточним все значения по количеству детей.

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Количество детей равное '-1' приравняем к '1', т.к. скорей всего предполагался "дефис".

In [58]:
data.loc[data['children'] < 0, 'children'] = 1

In [59]:
data['children'].value_counts()


0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [60]:
children_group = data[['children', 'debt']] 

def children_number(number):
    if number == 0:
        return 'детей нет'
    elif 1 <= number <= 2:
        return 'детей мало'
    return 'детей много'


children_group['children_count'] = children_group['children'].apply(children_number)
children_group

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,children,debt,children_count
0,1,0,детей мало
1,1,0,детей мало
2,0,0,детей нет
3,3,0,детей много
4,0,0,детей нет
...,...,...,...
21520,1,0,детей мало
21521,0,0,детей нет
21522,1,1,детей мало
21523,3,1,детей много


Посчитаем количество задолжностей по каждой группе, касаемо наличия детей.

In [61]:
children_group_new = children_group.groupby(['children_count']).agg({'debt':['count', 'sum']})

# Узнаем процент неплательщиков в каждой группе.
children_group_new['debt_ratio'] = children_group_new['debt']['sum'] / children_group_new['debt']['count']
children_group_new['debt_ratio'] = children_group_new['debt_ratio'].apply(lambda i: '{:.2%}'.format(i))

#ПЕреименуем столбцы для лучшего восприятия.
children_group_new.set_axis(['Всего клиентов', 'Задолжников', 'Процентная часть'], axis='columns', inplace=True)

children_group_new

,Всего клиентов,Задолжников,Процентная часть
children_count,,,
детей мало,6920,639,9.23%
детей много,456,39,8.55%
детей нет,14149,1063,7.51%


### Вывод

Выведенная таблица показывает, какую часть неплательщики каждой группы составляют от общего количество заемщиковв группе.

In [62]:
#children_group_new = (data.groupby(['children_count'])
#                          .agg({'children': 'count', 'debt':'sum'})
#                          .rename(columns={'children':'total'})
#                          .reset_idnex()
#                     )

#children_group_new['debt_ratio'] = (children_group['debt'] / children_group['total']) * 100
#children_group_new.round(2).sort_values('debt_ratio')

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [63]:
children_group_new

,Всего клиентов,Задолжников,Процентная часть
children_count,,,
детей мало,6920,639,9.23%
детей много,456,39,8.55%
детей нет,14149,1063,7.51%


### Вывод

Мы можем заметить зависимость невозврата кредита в срок от количества детей: малодетные клиенты являются самыми ненадежными плательщиками. С одной стороны количество детей должно говорить о стабильности заемщика, но возможно (т.к. дети являются дополнительным расходом) семьи с малым количеством детей не могут найти баланс между притоком и расходом денежных средств. К тому же у каждой семьи свой образ жизни: кто-то и с четырьмя детьми умудряется копить, а кто-то живет на широкую ногу, отдавая все одному ребенку, и ему денег не хватает. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [64]:
# Создадим сводную таблицу семейного положения и соответсвующее ему количество плательщиков и сумму должников.
family_group = data.groupby(['family_status']).agg({'debt':['count', 'sum']})


# Определим процент неплательщиков по каждой группе.
family_group['part'] = family_group['debt']['sum'] / family_group['debt']['count']
family_group['part'] = family_group['part'].apply(lambda i: '{:.2%}'.format(i))

# Переименуем столбцы для облегччения ориентирования.
family_group.set_axis(['Всего клиентов', 'Задолжников', 'Процентная часть'], axis='columns', inplace=True)
                        

family_group 


,Всего клиентов,Задолжников,Процентная часть
family_status,,,
в разводе,1195,85,7.11%
вдовец / вдова,960,63,6.56%
гражданский брак,4177,388,9.29%
женат / замужем,12380,931,7.52%
не женат / не замужем,2813,274,9.74%


### Вывод

Из таблицы видно, что клиенты состоящие в браке являются более дисциплинированными, в то время как холостые допускают больше "просрочек" по кредиту.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [65]:
# Создадим функция для определения достатка плательщика.
def prosperity(row):
    if 1 < row <= 50000:
        return 'клиент с низким достатком'
    elif 50000 < row <= 150000:
        return 'клиент со средним достатком'
    elif row > 150000:
        return 'клиент с высоким достатком'

#  Создадим на основе данных о доходе клиента столбец с уровнем достатка.    
data['prosperity'] = data['total_income'].apply(prosperity)

# Создадим таблицу с общим числом клиентов и числом неплательщиков на основе уровня доходов.
prosperity = data.groupby('prosperity')['debt'].agg(['count', 'sum'])

# Вычислим процентную составляющую.
prosperity['part'] = (prosperity['sum'] / prosperity['count'])

prosperity['part'] = prosperity['part'].apply(lambda i: '{:.2%}'.format(i))

prosperity.set_axis(['Всего клиентов', 'Задолжников', 'Процентная часть'], axis='columns', inplace=True)

In [66]:
prosperity.sort_values(by='Процентная часть', ascending=False)

,Всего клиентов,Задолжников,Процентная часть
prosperity,,,
клиент со средним достатком,11969,992,8.29%
клиент с высоким достатком,9184,726,7.91%
клиент с низким достатком,372,23,6.18%


### Вывод

Вывели интересные результаты: получилось, что "лидируют" по задолжностям в своем сегменте люди со средним достатком.Так же можем сделать вывод, что высокий доход не показатель возвращаемости кредита.

- Как разные цели кредита влияют на его возврат в срок?

Для начала получим уникальные значения для целей кредита. 

In [74]:
def lemma_purpose(row):
    if 'свадьба' in row:
        return 'свадьба'
    elif 'автомобиль' in row:
        return 'автомобиль'
    elif 'образование' in row:
        return 'образование'
    else:
        return 'недвижимость'
    

data['purpose_group'] = data['lemma_purpose'].apply(lemma_purpose)

In [75]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma_purpose,prosperity,purpose_group
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",клиент с высоким достатком,недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",клиент со средним достатком,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",клиент со средним достатком,недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",клиент с высоким достатком,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",клиент с высоким достатком,свадьба


In [80]:
table_purpose = data.groupby(['purpose_group'])['debt'].agg(['count', 'sum'])

table_purpose['debt_ratio'] = table_purpose['sum'] / table_purpose['count']

table_purpose['debt_ratio'] = table_purpose['debt_ratio'].apply(lambda i: '{:.2%}'.format(i))

table_purpose

,count,sum,debt_ratio
purpose_group,,,
автомобиль,4315,403,9.34%
недвижимость,10840,782,7.21%
образование,4022,370,9.20%
свадьба,2348,186,7.92%


### Вывод

Наибольшее количество должников по целям кредита в группе по автомобилям: многие хотят новую машину, но не многие способны на протяжении длительного периода тратить определенную сумму из своего бюджета (особенно при наличии одного ребенка и имея средний достаток). На втором месте - образование. Возможно должниками являются студенты.Меньше всего задолжностей в недвижимости. Такой разброс показателей говорит о том, что суммы на эти цели очень разняться и не всегда у заёмщиков быввают возможности погасить всё в срок.

### Шаг 4. Общий вывод

Результаты исследования показали, что есть наиболее явные зависимости по возврату задолжностей, а есть менее.
Видно, как влияет наличие детей - большое количество детей, лучшая возращаемость кредита. Так же явно прослеживается зависимоть от семейного положения - люди в браке более ответственны нежели холостые. 
Люди со средним достатком совершают больше просрочек, чем люди с низким. Возможно это объяснимо специальным завышением дохода для получения кредита.
Самая надежная цель получения кредита - недвижимость, а самая ненадежная - автомобиль. Возможно недвижимость подразумевает большую ответственность и как следствие наличие у людей некоторой финансовой "подушки". 
Таким образом мы выяснили, что все исследуемые факторы оказывают влияние на возврат кредита.